In [1]:
import requests
import os
import pandas as pd
from datetime import date
import time
import smtplib
from email.mime.text import MIMEText
from io import StringIO


hodeCanvas = {"Authorization": f'Bearer {os.environ["tokenCanvas"]}'}
idag = date.today().isoformat()[0:10]

In [2]:
url = "https://hvl.instructure.com/api/v1/accounts/1/sis_imports"
svar = requests.get(url, headers=hodeCanvas)
importer = svar.json()
antal_feilmeldingar = 0
innhald = ""
for i in importer['sis_imports']:
    if ("errors_attachment" in i) and (i['created_at'] > idag):
        fil = i["errors_attachment"]['url']
        data = pd.read_csv(StringIO(requests.get(fil, headers=hodeCanvas).content.decode("utf-8")))
        brukarar = data[data['message'].str.startswith('User not found')]
        eksisterer = data[data['message'].str.startswith('An existing')]
        for b in brukarar['message']:
            innhald += f"{b}\n"
            antal_feilmeldingar += 1
        for e in eksisterer['message']:
            innhald += f"{e}\n"
            antal_feilmeldingar += 1
print(innhald)
        

An existing Canvas user with the Canvas ID 48226 has already claimed fs:203:152758's user_id requested login information, skipping
An existing Canvas user with the Canvas ID 18488 has already claimed fs:203:677139's user_id requested login information, skipping
An existing Canvas user with the Canvas ID 23680 has already claimed fs:203:19319's user_id requested login information, skipping

